# Parte 1: Estadísticas para ingenieros que miran el cielo.

Dadas las señales de temperaturas diarias registradas durante cierto periodo en las tres ciudades (S1: Quito, S2: Melbourne, S3: Oslo), expresadas como valores enteros, en °C (grados centígrados):

1.1 Calcular la temperatura promedio y la desviación estándar para cada señal Si y analizar cómo se comportan estadísticamente.

1.2 Calcular el factor de correlación cruzada entre cada par de señales. Discutir si existen correlaciones significativas o no (tratando de establecer, por ejemplo, si Melbourne podría estar prediciendo el clima de Quito, o de Oslo.. o si no tienen nada que ver).

In [1]:
import math
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [93]:
#Se cargan los datos de la temperatura de las ciudades en datasets
dataset_raw_quito=pd.read_csv("temperature_Quito_celsius.csv")
dataset_raw_melbourne=pd.read_csv("temperature_Melbourne_celsius.csv")
dataset_raw_oslo=pd.read_csv("temperature_Oslo_celsius.csv")

Se identificaron outliers en los datasets de Melbourne y Oslo:

In [18]:
fig = px.box(dataset_raw_melbourne, x="AvgTemperature", width=800, height=400, title="Boxplot de Melbourne")

fig.show()

fig = px.box(dataset_raw_oslo, x="AvgTemperature", width=800, height=400, title="Boxplot de Oslo")

fig.show()

En ambos dataset, el valor de los outliers es el mismo: -73. Al momento de manejar este dato, se evaluaron distintas alternativas, tales como:

- Eliminar todas las entradas del dataset en las que aparezca dicha temperatura. Sin embargo, esto implicaría, para mantener consistencia, eliminar dichos días también de los demás datasets. Y, al estar realizadas las mediciones por días, perderlas dificultaría la toma de estadísticas entre años.

- "Reparar" los datos, a partir de sus datos vecinos. En la mayoría de las ocasiones en las que aparece un -73, lo hace entre dos valores cercanos, por ejemplo:

In [ ]:
# Mostrar aparición de -73 entre datos cercanos.

Entonces, la idea es completar ese dato a partir de los datos vecinos, calculando el promedio entre el vecino no outlier más cercano a izquierda y el más cercano a derecha.

In [51]:
datos_basura_melbourne = list(dataset_raw_melbourne[dataset_raw_melbourne["AvgTemperature"] == -73.0].index)
datos_basura_oslo = list(dataset_raw_oslo[dataset_raw_oslo["AvgTemperature"] == -73.0].index)

print("Indices de datos basura: ")
print(datos_basura_melbourne)
print(datos_basura_oslo)

Indices de datos basura: 
[1453, 1454, 1459, 1460, 1470, 2246, 2725, 2726, 2727, 2728, 2806, 2981, 3187, 4622, 4790, 5015, 5212]
[1453, 1454, 1459, 1460, 1470, 2725, 2726, 2727, 2728, 4622, 4723, 5212]


In [5]:
print("Número de datos en cada dataset: ")
print(len(dataset_quito))
print(len(dataset_melbourne))
print(len(dataset_oslo))

Número de datos en cada dataset: 
5844
5827
5832


Aproximadamente 16 años.

In [6]:
#Se obtiene, para cada dataset, una tabla con las temperaturas, su frecuencia y su probabilidad
temperaturas_melbourne=(dataset_melbourne["AvgTemperature"].value_counts()).to_frame().reset_index()
temperaturas_melbourne.columns = ["AvgTemperature", "count"]
temperaturas_melbourne["probabilidad"] = (temperaturas_melbourne["count"]/temperaturas_melbourne["count"].sum())

temperaturas_oslo=dataset_oslo["AvgTemperature"].value_counts().to_frame().reset_index()
temperaturas_oslo.columns = ["AvgTemperature", "count"]
temperaturas_oslo["probabilidad"] = (temperaturas_oslo["count"]/temperaturas_oslo["count"].sum())

temperaturas_quito=dataset_quito["AvgTemperature"].value_counts().to_frame().reset_index()
temperaturas_quito.columns = ["AvgTemperature", "count"]
temperaturas_quito["probabilidad"] = (temperaturas_quito["count"]/temperaturas_quito["count"].sum())

In [7]:
def calcular_promedio(dataset):
    sum = 0
    for temperature in dataset["AvgTemperature"]:
        sum = sum + temperature
    return sum / len(dataset["AvgTemperature"])

In [8]:
def calcular_prom(dataset):
    sum = 0
    for temperature in dataset["AvgTemperature"]:
        sum = sum + (temperature *temperaturas_melbourne[temperaturas_melbourne["AvgTemperature"] == temperature]["probabilidad"].values[0])
    return sum

In [9]:
def calcular_prom2(dataset):
    # Se pondera cada temperatura por su probabilidad.
    # Luego, se realiza la suma de cada 
    return (dataset["AvgTemperature"] * dataset["probabilidad"]).sum()

Promedios de cada ciudad:

In [10]:
promedio_melbourne = calcular_promedio(dataset_melbourne)
prom = calcular_prom(temperaturas_melbourne)
prom2 = calcular_prom2(temperaturas_melbourne)
promedio_oslo = calcular_promedio(dataset_oslo)
promedio_quito = calcular_promedio(dataset_quito)

print(promedio_melbourne)
print(prom)
print(prom2)
print(promedio_oslo)
print(promedio_quito)

17.798009267204392
17.798009267204392
17.798009267204392
4.770919067215363
13.603524982888432


In [11]:
def calcular_desvio_estandar(dataset,promedio):
    sum = 0
    for temperature in dataset["AvgTemperature"]:
        sum += pow(temperature - promedio,2)
    return np.sqrt(sum/len(dataset["AvgTemperature"]-1)) 

In [12]:
desvio_quito = calcular_desvio_estandar(dataset_quito, promedio_quito)
desvio_melbourne = calcular_desvio_estandar(dataset_melbourne, promedio_melbourne)
desvio_oslo = calcular_desvio_estandar(dataset_oslo, promedio_oslo)

print(desvio_quito)
print(desvio_melbourne)
print(desvio_oslo)

1.3015790563505123
4.253573526810525
8.790504407735737


Quito: Donde la temperatura no cambia ni aunque recen diez climas distintos.
Melbourne: Donde podés experimentar las cuatro estaciones antes del almuerzo.
Oslo: Donde el clima no se decide si quiere ser Siberia o un spa nórdico.

Siberia -> Temperaturas extremadamente bajas, mucha nieve.

In [13]:
# Se obtiene el dataset conjunto de las temperaturas de dos ciudades, junto con la frecuencia del par
def obtener_dataset_conjunto(dataset1, dataset2):
    resultado = pd.DataFrame({
        "temperaturas_ciudad1": dataset1["AvgTemperature"].values,
        "temperaturas_ciudad2": dataset2["AvgTemperature"].values
    }).value_counts().to_frame().reset_index()
    resultado.columns = ["temperaturas_ciudad1", "temperaturas_ciudad2", "frecuencia"]
    return resultado

In [14]:
def obtener_probabilidad_conjunta(primer_temperatura, segunda_temperatura, dataset):

    filtro = (
        (dataset["temperaturas_ciudad1"] == primer_temperatura) &
        (dataset["temperaturas_ciudad2"] == segunda_temperatura)
    )

    resultado = dataset[filtro]["frecuencia"] / dataset["frecuencia"].sum()

    if (len(resultado) == 0):
        return 0
    else:
        return resultado.values[0]

In [15]:
print(obtener_probabilidad_conjunta(21.0, 0.0, obtener_dataset_conjunto(dataset_melbourne, dataset_oslo)))

ValueError: All arrays must be of the same length

In [ ]:
calcular_promedio_conjunto(first_dataset, second_dataset):
    sum = 0
    


<img src="image-20250611-104115.png" width="" align="" />

La correlación cruzada, en términos generales, es una medida de la similitud entre dos señales o procesos, considerando un posible desfase temporal entre ellos. Se utiliza para identificar si hay una relación entre dos señales y, en caso afirmativo, determinar el grado de esa relación y el tiempo de desfase

El coeficiente de correlación cruzada (o simplemente correlación cruzada) es una medida estadística que cuantifica la similitud entre dos series temporales o señales. Indica la fuerza y dirección de la relación entre ellas, incluyendo un desfase temporal (retardo). Varía entre -1 y +1, donde -1 indica una correlación negativa perfecta, 0 ninguna correlación y +1 una correlación positiva perfecta. 

# Parte 2: Una fuente de calor… markoviana

Considerando los valores de temperatura t que componen cada señal Si, construir una nueva señal Ti  compuesta por una secuencia de símbolos discretos F, T o C, definidos según: 
- F (frío): si t < 11°C
- T (templado): si 11 ≤ t < 19°C
- C (cálido): si t ≥ 19°C
Para cada Ti:
Modelar la fuente con memoria de orden 1 (Markov), obtener la matriz de transición y analizar su comportamiento (por ejemplo, tratá de descubrir cosas como: En Oslo, si hace frío hoy, es casi seguro que siga así hasta julio..)
Usar muestreo Monte Carlo para obtener, para cada símbolo:
La probabilidad estacionaria (esa a la que llegás después de mucho simular).
El tiempo medio de 1° recurrencia (ese que te dice, en promedio, cuánto tarda un símbolo en volver a aparecer después de haberse emitido).
Nota: Experimentar con diferentes umbrales de convergencia ε (comentar si realmente influyen en los resultados, o todo es una ilusión matemática). Analizar precisión de resultados en función del tiempo e incluir gráfico de convergencia.

In [126]:
def obtener_fuente_discreta(dataset):
    return dataset.map(lambda x: 'F' if x < 11.0 else 'T' if x < 19.0 else 'C')

print(obtener_fuente_discreta(dataset_raw_melbourne))

     AvgTemperature
0                 C
1                 T
2                 C
3                 C
4                 C
...             ...
5839              T
5840              C
5841              C
5842              C
5843              C

[5844 rows x 1 columns]


In [129]:
dataset_raw_melbourne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AvgTemperature  5844 non-null   float64
dtypes: float64(1)
memory usage: 45.8 KB


In [ ]:
def get_matriz_transicion(nro_simbolos, secuencia):

    matriz_transicion = [[0 * nro_simbolos] * nro_simbolos]

    simbolo = secuencia[0]
    for siguiente_simbolo in secuencia[1:]:
        matriz_transicion[simbolo][simbolo_siguiente] += 1

    simbolos_emitidos = len(secuencia)
    for i in range(simbolos_emitidos):
        for j in range(simbolos_emitidos):
            if (matriz_transicion[i][j] > 0):
                matriz_transicion[i][j] = (matriz_transicion[i][j] / simbolos_emitidos)

    return matriz_transicion

print(get_matriz_transicion(3, ))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1ebd8bcc-2607-4fac-b013-6f5656f47292' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>